Opdracht: Inleveropdracht 4: Machine Translation & Document Search 
Naam: Onno de Jong <br>
Studentnummer: 1809878

In [1]:
#imports
import numpy as np
import matplotlib.pyplot as plt
import pickle
import math

import pandas as pd

Embeddings inladen

In [2]:
en, fr = pickle.loads(open('data/en_embeddings.p', 'rb').read()), pickle.loads(open('data/fr_embeddings.p', 'rb').read())

inladen train en test sets

In [3]:
train = pd.read_csv('data/en-fr.train.txt',sep=" ", names=['en', 'fr'])
test =  pd.read_csv('data/en-fr.test.txt',sep=" ", names=['en', 'fr'])

In [4]:
train.sample(5)

,en,fr
8683,demolished,démolie
2470,agree,acceptez
8148,hebrew,hébreux
7194,opinions,opinions
5494,escape,échapper


In [5]:
test.sample(5)

,en,fr
676,zoo,zoo
2562,gibraltar,gibraltar
1988,electron,électron
1920,varied,variés
2609,maya,mayas


cleaning van de sets, er zijn een heel aantal worden, vooral bij de franse woorden, die niet in de embeddings zitten, die haal ik er uit omdat ik er geen informatie van weet

In [6]:
sum([x in fr.keys() for x in train['fr']]) / len(train)

0.5275938189845475

In [7]:
sum([x in en.keys() for x in train['en']]) / len(train)

0.9908020603384842

In [8]:
print(train.shape)
clean_train = train[train['en'].isin(en.keys())]
clean_train = clean_train[clean_train['fr'].isin(fr.keys())]
print(clean_train.shape)

(10872, 2)
(5717, 2)


In [9]:
print(test.shape)
clean_test = test[test['en'].isin(en.keys())]
clean_test = clean_test[clean_test['fr'].isin(fr.keys())]
print(clean_test.shape)

(2943, 2)
(1542, 2)


checken of alle woorden nu aanwezig zijn in de embeddings

In [10]:
sum([x in fr.keys() for x in clean_train['fr']]) / len(clean_train)

1.0

In [11]:
sum([x in en.keys() for x in clean_train['en']]) / len(clean_train)

1.0

Helemaal compleet

In [12]:
def gradient_descrent(X: np.ndarray, Y: np.ndarray, num_iter: int, lr: float)-> tuple[np.ndarray, float]:
    """
    Een implementatie van het gradient descent algoritme op de gradient van de frobenius norm
    
    Params:
    --------
    X: np.ndarray
        Vector waar de Engelse embeddings in staan.
    Y: np.ndarray
        Vector waar de Franse embeddings in staan.
    num_iter: int
        Het aantal iteraties dat het algoritme uitvoert.
    lr: float
        De learning rate van de gradient descent.
    
    Returns:
    --------
    np.ndarray
        Het model.
    float
        De loss aan het einde van het trainen van de weights.
    """
    m, n = X.shape
    R = np.zeros((n,n))
    for _ in range(num_iter):
        gradient = 2/m * np.dot(X.T, np.dot(X, R) - Y)
        R -= gradient * lr
    return R, 1/m * ((np.dot(X, R) - Y)**2).sum()

In [13]:
def get_embeddings(n, tset):
    res = []
    for en_word, fr_word in tset[['en', 'fr']].values[:n]:
        res.append([en[en_word], fr[fr_word]])
        
    return np.array(res)

In [14]:
t = get_embeddings(len(clean_train), clean_train)

In [15]:
t[:, 0].shape

(5717, 300)

In [16]:
model, loss = gradient_descrent(t[:, 0], t[:, 1], 400, .8)

In [17]:
model.shape

(300, 300)

Cosine distance functie

In [ ]:
def cosine_dist(A : np.ndarray, B : np.ndarray) -> float:
    """
    Een implementatie van de cosine formule.
    
    A: np.ndarray
        Vector A.
    B: np.ndarray
        Vector B.
    """
    
    # return np.dot(A,B)/(np.linalg.norm(A)*np.linalg.norm(B))
    return np.dot (A, B) / (np.sqrt(np.sum(A ** 2)) * np.sqrt(np.sum(B ** 2)))

In [19]:
def predict(en_word: str, en_words: dict[str, np.ndarray], fr_words: dict[str, np.ndarray], R: np.ndarray):
    """
    De prediction functie die werkt met het getrainde model.

    Params:
    --------
    en_word: str
        Het engelse woord dat vertaald moet worden.
    en_words: dict[str, np.ndarray]
        De dictionary van embeddings van engelse woorden.
    fr_words: dict[str, np.ndarray]
        De dictionary van embeddings van franse woorden.
    R: np.ndarray
        Het model.

    Returns:
    --------
    string
        het voorspelde franse woord.
    """
    
    target = np.matmul(en_words[en_word], R)
    closest = [-1, ""]
    for index, embedding in fr_words.items():
        c_dist = cosine_dist(embedding, target)
        if c_dist > closest[0]:
            closest = [c_dist, index]
    return closest[1]

In [20]:
def accuracy(word_dict: pd.DataFrame, en_words: dict[str, np.ndarray], fr_words: dict[str, np.ndarray], R: np.ndarray):
    """
    Een simpele accuracy functie die de accuracy berekent van een model over een aantal woorden.

    Params:
    --------
    word_dict: pd.DataFrame
        Een dataframe met de kolommen "en" en "fr" die van engels naar frans vertaald moeten worden.
    en_words: dict[str, np.ndarray]
        De dictionary van embeddings van engelse woorden.
    fr_words: dict[str, np.ndarray]
        De dictionary van embeddings van franse woorden.
    R: np.ndarray
        Het model.

    Returns:
    --------
    float
        De accuracy van het model.
    """
    
    total = 0
    count = 0
    for en_word, fr_word in word_dict[['en', 'fr']].values:
        count += 1
        # print(count) if count % 100 == 0 else None
        if predict(en_word, en_words, fr_words, R) == fr_word:
            total += 1
#         print(i, fr_word)
    return total / len(word_dict)

Een ~43% accuracy op de test set en een ~55% accuracy op de train set. 

In [21]:
accuracy(clean_test, en, fr, model)

0.43190661478599224

In [22]:
accuracy(clean_train, en, fr, model)

0.5460906069616932

## Deel II: Document Search

In [42]:
list(en.keys())[:50]

['the',
 'was',
 'for',
 'that',
 'with',
 'from',
 'this',
 'utc',
 'his',
 'not',
 'are',
 'talk',
 'which',
 'also',
 'were',
 'but',
 'have',
 'one',
 'new',
 'first',
 'page',
 'you',
 'they',
 'had',
 'article',
 'who',
 'all',
 'their',
 'there',
 'made',
 'its',
 'people',
 'may',
 'after',
 'other',
 'should',
 'two',
 'score',
 'her',
 'can',
 'would',
 'more',
 'she',
 'when',
 'time',
 'team',
 'american',
 'such',
 'discussion',
 'links']

Function to clean documents

In [24]:
from nltk import PorterStemmer, RegexpTokenizer
from nltk.corpus import stopwords
import regex as re
import string
import numpy as np

ps = PorterStemmer()
regexp = RegexpTokenizer(r'\w+|\$[\d\.]+|\S+')

with open("data/iliad.txt", "r", encoding="utf-8") as f:
    iliad = f.read()
with open("data/odyssey.txt", "r", encoding="utf-8") as f:
    odyssey = f.read()

In [25]:
iliad[:100]

'The Project Gutenberg eBook of The Iliad\n    \nThis ebook is for the use of anyone anywhere in the Un'

In [26]:
badwords = stopwords.words("english") + list(string.punctuation)

def clean_document(document):
    # verwijzingen geven geen extra informatie
    document = re.sub("[\[(].+[\])]", '', document)

    # Tokenizer, sentences -> words
    cleaned_document = regexp.tokenize(document)
    cleaned_document_full = ""

    if len(cleaned_document) > 2: # here we rebuild the cleaned document back to a clean full sentence to show later. This is not used for anything besides showing the sample
        cleaned_document_full += cleaned_document[0]

        for word in cleaned_document[1:]:
            if word not in string.punctuation:
                cleaned_document_full += " "
            
            cleaned_document_full += word

    # stopwords en punctuation
    document = [word for word in cleaned_document if word not in badwords]

    # stemming words, Ik stem hier de woorden expliciet niet omdat veel gestemde woorden niet voorkomen in de woord embeddings
    # documents = [[ps.stem(x) for x in document] for document in documents]

    return document, cleaned_document_full

def clean_documents(document: str):
    # split documents
    documents = re.split("[.:;!?]", document)
 
    return list(zip(*map(clean_document, documents)))

In [27]:
clean_docs, og_docs = {}, {}

clean_docs["iliad"], og_docs["iliad"] = clean_documents(iliad)
clean_docs["odyssey"], og_docs["odyssey"] = clean_documents(odyssey)


print("partially cleaned sample:\t", og_docs["iliad"][63])
print("Cleaned sample:\t\t\t", clean_docs["iliad"][63])
actual_words = set(clean_docs["iliad"][63]).intersection(set(en.keys()))
print()

print("Words there are embeddings for", actual_words)
print("% amount of words in cleaned sample:",len(actual_words) / len(clean_docs["iliad"][63]))

partially cleaned sample:	 To form correct views of individuals we must regard them as forming parts of a great whole —we must measure them by their relation to the mass of beings by whom they are surrounded, and, in contemplating the incidents in their lives or condition which tradition has handed down to us, we must rather consider the general bearing of the whole narrative, than the respective probability of its details
Cleaned sample:			 ['To', 'form', 'correct', 'views', 'individuals', 'must', 'regard', 'forming', 'parts', 'great', 'whole', '—we', 'must', 'measure', 'relation', 'mass', 'beings', 'surrounded', 'contemplating', 'incidents', 'lives', 'condition', 'tradition', 'handed', 'us', 'must', 'rather', 'consider', 'general', 'bearing', 'whole', 'narrative', 'respective', 'probability', 'details']

Words there are embeddings for {'condition', 'respective', 'incidents', 'consider', 'relation', 'surrounded', 'details', 'great', 'mass', 'probability', 'correct', 'rather', 'views',

Function to calculate document embedding

In [28]:
def calculate_document_embedding(document: list[str]):
    return sum([en.get(word, 0) for word in document]) # De embeddings optellen van alle woorden in het document

def calculate_document_embeddings(documents: dict[str, list[list[str]]], og_docs):
    embeddings = []
    docs = {}

    index = 0
    for key, value in documents.items():
        print(key)
        for id, document in enumerate(value):
            embedding = calculate_document_embedding(document) 
            if np.any(embedding):
                # ik maak 2 aparte lijsten met het document en de embedding omdat geen van beide de key van een dict kan zijn. het id van het document kan worden gebruikt om de embedding op te vragen
                embeddings.append(embedding)
                docs[index] = [og_docs[key][id], key]
                index += 1

    return embeddings, docs

In [29]:
embedding_database, document_database = calculate_document_embeddings(clean_docs, og_docs)

iliad
odyssey


In [30]:
document_database[0]

['The Project Gutenberg eBook of The Iliad This ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever',
 'iliad']

Nummer 63 is nu een ander document omdat er een aantal documenten voor zaten die waar geen embedding voor beschikbaar was

In [31]:
len(embedding_database)

18703

18723 samples, als we ongeveer gemiddeld 10 samples per bucket willen dan hebben we log2(1872) ~= 11 hyperplanes nodig

Function to hash documents


In [32]:
def create_planes(n_planes, n_dimensions):
    return np.random.normal(size=[n_dimensions, n_planes])

def hash_vector(vec, planes):
    dot = np.dot(vec, planes)

    sign_of_dot = np.sign(dot)

    h = sign_of_dot >= 1

    hash_bucket = "".join(map(str, map(int, h))) # Translate it to binary representation

    return hash_bucket

In [33]:
def get_all_binary(n):
    strings = []
    for i in range(2**n):
        binary_str = format(i, '0' + str(n) + 'b') # omzetten van int naar binary representatie
        strings.append(binary_str)
    
    return strings

def create_buckets(n_planes):
    buckets = {}
    for x in get_all_binary(n_planes): # De keys van de buckets zijn een representatie van de planes
        buckets[x] = []

    return buckets

def populate_buckets(embedding_database, document_database, planes):
    buckets = create_buckets(planes.shape[1])

    for id, _ in document_database.items():
        buckets[hash_vector(embedding_database[id], planes)].append(id) # hash vector to the correct bucket

    return buckets

In [34]:
p = create_planes(11, 300) # Random planes creeren
hash_table = populate_buckets(embedding_database, document_database, p) # Hash table bouwen

In [35]:
dict(list(hash_table.items())[:5])

{'00000000000': [1800, 5970, 10901],
 '00000000001': [4671,
  5376,
  5842,
  8397,
  8587,
  12403,
  12447,
  12801,
  13576,
  13998,
  16219,
  17264],
 '00000000010': [1386, 2672, 3864, 7024, 10019, 12477, 12520],
 '00000000011': [1145,
  1394,
  2076,
  2115,
  2142,
  2971,
  4208,
  4258,
  4686,
  5652,
  6455,
  6567,
  6603,
  6866,
  7207,
  7342,
  8044,
  9508,
  11208,
  11919,
  13658,
  15293,
  15440,
  15697,
  15904,
  17867,
  18220],
 '00000000100': [740,
  986,
  1592,
  1734,
  3745,
  4337,
  5085,
  6286,
  7448,
  9885,
  9984,
  10519,
  10864,
  10965,
  12111,
  12716,
  13932]}

Er is geen sprake van een even verdeling van de sample. Dit was ook niet de verwachting

KNN algoritme

In [36]:
def KNN(samples, target, n, distance_func=cosine_dist): # Simpele inplementatie van een KNN algoritme (Ik weet niet zeker of ik het helemaal correct heb uitgewerkt op deze manier)
    s = [[x, distance_func(target, embedding_database[x])] for x in samples]
    s = list(sorted(s, key=lambda x: x[1], reverse=True))

    if len(s) > n:
        return s[:n]
    
    return s

In [37]:
def predict(sentence, hash_table, n):
    clean_sentence = clean_document(sentence)[0]
    embed_sentence = calculate_document_embedding(clean_sentence)

    hash = hash_vector(embed_sentence, p)
    bucket = [document_database[id] for id in hash_table[hash]]
    knn_res = KNN(hash_table[hash], embed_sentence, n)
    knn_res = [document_database[x[0]] for x in knn_res] # Convert naar originele samples

    books = {}
    for doc in knn_res:
        if doc[1] in books:
            books[doc[1]] += 1
            continue
        books[doc[1]] = 1
    book = sorted(list(books.items()), key=lambda x: x[1])[0][0]

    return bucket, knn_res, book

Ter voorbeeld dat het allemaal werkt heb ik een passage uit iliad gepakt. Je ziet dat het algoritme de identieke passage vindt en daarbij een aantal andere passages die erop lijken

In [38]:
s = """and thus the chief of Troy replied"""
bucket, knn_res, book = predict(s, hash_table, 5)
print("N samples in bucket:", len(bucket))
print("Most similar samples:", knn_res)
print("This sentence is most similar to:", book)

N samples in bucket: 8
Most similar samples: [['and thus the chief of Troy replied', 'iliad'], ['and thus the chief replies', 'iliad'], ['The towering Ajax, with an ample stride, Advanced the first, and thus the chief defied', 'iliad'], ['whom, with fury moved, Opprobrious thus, th ’ impatient chief reproved', 'iliad'], ['” So parts the chief', 'iliad']]
This sentence is most similar to: iliad


In [39]:
s = """“My friend,” answered Nestor, “you recall a time of much sorrow to my
mind, for the brave Achaeans suffered much both at sea, while
privateering under Achilles, and when fighting before the great city of king Priam"""
bucket, knn_res, book = predict(s, hash_table, 5)
print("N samples in bucket:", len(bucket))
print("Most similar samples:", knn_res)
print("This sentence is most similar to:", book)

N samples in bucket: 439
Most similar samples: [['” “My friend ,” answered Nestor, “you recall a time of much sorrow to my mind, for the brave Achaeans suffered much both at sea, while privateering under Achilles, and when fighting before the great city of king Priam', 'odyssey'], ['he was running up fast into manhood, and bade fare to be no worse man, face and figure, than his father, but some one, either god or man, has been unsettling his mind, so he has gone off to Pylos to try and get news of his father, and the suitors are lying in wait for him as he is coming home, in the hope of leaving the house of Arceisius without a name in Ithaca', 'odyssey'], ['We know what fate befell each one of the other heroes who fought at Troy, but as regards Ulysses heaven has hidden from us the knowledge even that he is dead at all, for no one can certify us in what place he perished, nor say whether he fell in battle on the mainland, or was lost at sea amid the waves of Amphitrite', 'odyssey'], ['